## MS + CO WD

### Purpose

Mimic a merger between a cool CO WD and a main-sequence star.

In [1]:
from qol.mesa.launcher import *
import qol.info as info

In [2]:
# Goal: reproduce something similar to m_plus_wd

# Presets
net_name = 'cno_extras_o18_to_mg26.net'
MPWD_in_Msun = 3. # progenitor mass of WD (difficult to control WD mass directly)
MMS_in_Msun = 0.5 # MS mass
T_WD = 8000 # WD temperature
ringdown_time_yr = 1e5

### Step 1: Create a CO WD

massive-enough progenitor to avoid He flash

In [3]:
inlist = MesaInlist('inlist_make_hot_co_wd', LOGS_dir='LOGS/make_hot_co_wd/')
inlist.enable_pgstar()
inlist.save_pgstar(write_path='Grid1/hot_co_wd/')
inlist.use_qol_pgstar()

# initial model
inlist.initial_mass(MPWD_in_Msun)
inlist.set_Zbase(0.02)
inlist.change_net(net_name)

inlist.he_core_boundary_h1_fraction(1e-3)
inlist.co_core_boundary_he4_fraction(1e-3)

inlist.energy_eqn_option('eps_grav')

# wind
inlist.cool_wind_RGB(scheme='Reimers', scaling_factor=0.5)
inlist.cool_wind_AGB(scheme='Blocker', scaling_factor=0.1)
inlist.RGB_to_AGB_wind_switch(1e-4)
inlist.cool_wind_full_on_T(9.99e9) # force usage of cool wind only
inlist.hot_wind_full_on_T(1.e10)

# stop at WD
inlist.stop_at_phase_WDCS()
inlist.save_final_model('hot_co_wd.mod')

task_make_hot_co_wd = inlist

### Step 2: cool CO WD

In [4]:
inlist = MesaInlist('inlist_cool_co_wd', LOGS_dir='LOGS/cool_co_wd/')
inlist.enable_pgstar()
inlist.save_pgstar(write_path='Grid1/cool_co_wd/')
inlist.use_qol_pgstar()

inlist.load_model('hot_co_wd.mod')
inlist.he_core_boundary_h1_fraction(1e-3)
inlist.co_core_boundary_he4_fraction(1e-3)

inlist.set_Zbase(0.02)

inlist.energy_eqn_option('eps_grav')

# resolution
inlist.min_dq(1e-25)
inlist.max_surface_cell_dq(1e-18)

# wind
inlist.cool_wind_RGB(scheme='Reimers', scaling_factor=0.5)
inlist.cool_wind_AGB(scheme='Blocker', scaling_factor=0.1)
inlist.RGB_to_AGB_wind_switch(1e-4)
inlist.cool_wind_full_on_T(9.99e9) # force usage of cool wind only
inlist.hot_wind_full_on_T(1.e10)

# mixing
inlist.use_Ledoux_criterion()
inlist.alpha_semiconvection(4e-2)
inlist.thermohaline_coeff(2.)

inlist.gravitational_settling(diffusion_class_representatives=['h1', 'he3', 'he4', 'c12', 'o16', 'ne20', 'ne22', 'mg26'],
        diffusion_use_cgs_solver=True,
        show_diffusion_info=True,
        diffusion_steps_hard_limit=2000,
        diffusion_maxsteps_for_isolve=2000)

# stop after cooled to desired temperature
inlist.Teff_lower_limit(T_WD)
inlist.save_final_model('cool_co_wd.mod')

task_cool_co_wd = inlist

### Step 3: create envelope matching core model

In [5]:
script = MesaPythonScript('inner_bc.py',
            template=f'{info.qol_path}mesa/templates/scripts/call_create_env_inlist_from_core.py',
              const_args=[MMS_in_Msun], prereqs=['cool_co_wd.mod'], products=['inlist_env_inner_bc'])

task_inner_bc = script

### Step 4: run envelope to th eq

In [6]:
inlist = MesaInlist('inlist_env_to_th_eq', LOGS_dir='LOGS/env_to_th_eq/')
inlist.enable_pgstar()
inlist.save_pgstar(write_path='Grid1/env_to_th_eq/')
inlist.use_qol_pgstar()

# set inner boundary condition
inlist.read_extra_inlist(namelist='star_job', rel_path='inlist_env_inner_bc', category='relax inner BC to accommodate core model')

# initialize as pre-MS
inlist.create_pre_main_sequence_model(True)
inlist.initial_mass(MMS_in_Msun)
inlist.initial_y(0.28)
inlist.initial_z(0.02)
inlist.set_Zbase(0.02)
inlist.change_net(net_name)

# relax some convergence conditions, disable dx/dt from burning
inlist.min_timestep_limit(1e-12)
inlist.energy_eqn_option('dedt')
inlist.use_gold_tolerances(False)
inlist.convergence_ignore_equL_residuals(True)
inlist.set_max_num_retries(3000)
inlist.limit_for_rel_error_in_energy_conservation(-1.)
inlist.disable_dxdt_from_nuclear_burning()

inlist.max_age(1e10)
inlist.save_final_model('env_th_eq.mod')

task_env_to_th_eq = inlist

### Step 5: stitch core and envelope together

In [7]:
script = MesaPythonScript(rel_path='merge.py',
            template=f'{info.qol_path}mesa/templates/scripts/call_create_shell_burning_remnant.py',
            prereqs=['cool_co_wd.mod', 'env_th_eq.mod'],
            products=['remnant_init.mod'])

task_merge = script

### Step 6: evolve remnant into HSE

In [8]:
inlist = MesaInlist('inlist_remnant_ringdown', LOGS_dir='LOGS/remnant_ringdown/')
inlist.enable_pgstar()
inlist.save_pgstar(write_path='Grid1/remnant_ringdown/')
inlist.use_qol_pgstar()

inlist.load_model('remnant_init.mod')
inlist.set_Zbase(0.02)

# enable hydro with drag
# still disable dx/dt from burning
inlist.enable_hydrodynamics()
inlist.add_drag_for_HSE(drag_coefficient=1.)
inlist.energy_eqn_option('eps_grav')
inlist.min_timestep_limit(1e-12)
inlist.use_gold_tolerances(False)
inlist.convergence_ignore_equL_residuals(True)
inlist.disable_dxdt_from_nuclear_burning()
inlist.disable_mixing()

# set arbitrary ringdown time
inlist.max_age(ringdown_time_yr)
inlist.save_final_model('remnant_hse.mod')

task_remnant_ringdown = inlist

### Step 7: evolve merger remnant to WD again

In [9]:
inlist = MesaInlist('inlist_remnant_to_co_wd', LOGS_dir='LOGS/remnant_to_co_wd/')
inlist.enable_pgstar()
inlist.save_pgstar(write_path='Grid1/remnant_to_co_wd/')
inlist.use_qol_pgstar()

inlist.load_model('remnant_hse.mod')
inlist.he_core_boundary_h1_fraction(1e-3)
inlist.co_core_boundary_he4_fraction(1e-3)

inlist.set_Zbase(0.02)

inlist.energy_eqn_option('eps_grav')

# wind
inlist.cool_wind_RGB(scheme='Reimers', scaling_factor=0.5)
inlist.cool_wind_AGB(scheme='Blocker', scaling_factor=0.1)
inlist.RGB_to_AGB_wind_switch(1e-4)
inlist.cool_wind_full_on_T(9.99e9) # force usage of cool wind only
inlist.hot_wind_full_on_T(1.e10)

# stop at WD
inlist.stop_at_phase_WDCS()
inlist.save_final_model('hot_remnant_co_wd.mod')

task_remnant_to_co_wd = inlist

### Step 8: evolve WD as long as possible

In [10]:
inlist = MesaInlist('inlist_cool_remnant_co_wd', LOGS_dir='LOGS/cool_remnant_co_wd/')
inlist.enable_pgstar()
inlist.save_pgstar(write_path='Grid1/cool_remnant_co_wd/')
inlist.use_qol_pgstar()

inlist.load_model('hot_remnant_co_wd.mod')
inlist.he_core_boundary_h1_fraction(1e-3)
inlist.co_core_boundary_he4_fraction(1e-3)

inlist.set_Zbase(0.02)

inlist.energy_eqn_option('eps_grav')

# resolution
inlist.min_dq(1e-25)
inlist.max_surface_cell_dq(1e-18)

# wind
inlist.cool_wind_RGB(scheme='Reimers', scaling_factor=0.5)
inlist.cool_wind_AGB(scheme='Blocker', scaling_factor=0.1)
inlist.RGB_to_AGB_wind_switch(1e-4)
inlist.cool_wind_full_on_T(9.99e9) # force usage of cool wind only
inlist.hot_wind_full_on_T(1.e10)

# mixing
inlist.use_Ledoux_criterion()
inlist.alpha_semiconvection(4e-2)
inlist.thermohaline_coeff(2.)

inlist.gravitational_settling(diffusion_class_representatives=['h1', 'he3', 'he4', 'c12', 'o16', 'ne20', 'ne22', 'mg26'],
        diffusion_use_cgs_solver=True,
        show_diffusion_info=True,
        diffusion_steps_hard_limit=2000,
        diffusion_maxsteps_for_isolve=2000)

# stop after a long time, if needed... okay to fail here, if sufficiently cooled
inlist.max_age(1e10)
inlist.save_final_model('cool_remnant_co_wd.mod')

task_cool_remnant_co_wd = inlist

In [11]:
work = MesaWorkingDirectory(run_path='/Users/nrui/Desktop/ms_plus_cowd/')
work.copy_history_columns_list(f'{info.qol_path}mesa/resources/r24.08.1/history_columns.list')
work.copy_profile_columns_list(f'{info.qol_path}mesa/resources/r24.08.1/profile_columns.list')
work.load_qol_pgstar()

work.add_task(task_make_hot_co_wd)
work.add_task(task_cool_co_wd)
work.add_task(task_inner_bc)
work.add_task(task_env_to_th_eq)
work.add_task(task_merge)
work.add_task(task_remnant_ringdown)
work.add_task(task_remnant_to_co_wd)
work.add_task(task_cool_remnant_co_wd)

work.save_directory(grant_perms=True)